This is an experiment to try to map the areas of Austin, TX where a grocery 
store can be reached by at most 15 minutes of walking.

The first iteration will focus on Central and East Austin.

This is a patchwork based off of various blog posts, library examples and 
Stackoverflow posts.

Here are the main resources:
- https://medium.com/@cheng.cesar/15-min-city-8c00dbe279fc
- https://geoffboeing.com/2017/08/isochrone-maps-osmnx-python/
- https://github.com/gboeing/osmnx-examples/blob/main/notebooks/13-isolines-isochrones.ipynb
- https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.geometries
- https://stackoverflow.com/questions/71268239/how-do-i-plot-multiple-isochrones-polygons-using-python-osmnx-library

In [ ]:
import osmnx as ox
import pandas as pd
import networkx as nx
import geopandas as gpd
import shapely
from shapely.geometry import Point
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [ ]:
# function to get isochrones
def get_isochrone(
    lon,
    lat,
    walk_times,
    speed=4.5,
    name=None,
    point_index=None,
):
    """Get isochrones."""
    loc = (lat, lon)
    G = ox.graph_from_point(loc, simplify=True, network_type="walk")
    # gdf_nodes = ox.graph_to_gdfs(G, edges=False)
    center_node = ox.distance.nearest_nodes(G, lon, lat)

    meters_per_minute = speed * 1000 / 60  # km per hour to m per minute
    for u, v, k, data in G.edges(data=True, keys=True):
        data["time"] = data["length"] / meters_per_minute
    polys = []
    for walk_time in walk_times:
        subgraph = nx.ego_graph(G, center_node, radius=walk_time, distance="time")
        node_points = [
            Point(data["x"], data["y"]) for node, data in subgraph.nodes(data=True)
        ]
        polys.append(gpd.GeoSeries(node_points).unary_union.convex_hull)
    info = {}
    if name:
        info["name"] = [name for t in walk_times]
    if point_index:
        info["point_index"] = [point_index for t in walk_times]
    return {**{"geometry": polys, "time": walk_times}, **info}


In [ ]:
# Settings
ox.settings.use_cache = True
ox.settings.log_console=False

In [ ]:
# Load the list of grocery stores and convert it to a GeoDataFrame.
df = pd.read_csv("grocery-stores.csv")
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["Longitude"], df["Latitude"]), crs="EPSG:4326")

In [ ]:
walk_times = [5, 10, 15]
isochrones = pd.concat(
    [
        gpd.GeoDataFrame(
            get_isochrone(
                r["geometry"].x,
                r["geometry"].y,
                name=r["Name"],
                point_index=i,
                walk_times=walk_times,
            ),
            crs=gdf.crs,
        )
        for i, r in gdf.iterrows()
    ]
)

# Build the map.
gdf = isochrones.set_index(["time", "point_index"]).copy()
# remove shorter walk time from longer walk time polygon to make folium work better
for idx in range(len(walk_times) - 1, 0, -1):
    gdf.loc[walk_times[idx], "geometry"] = (
        gdf.loc[walk_times[idx]]
        .apply(
            lambda r: r["geometry"].symmetric_difference(
                gdf.loc[(walk_times[idx - 1], r.name), "geometry"]
            ),
            axis=1,
        )
        .values
    )

m = gdf.reset_index().explore(column="time", height=1600, scheme="boxplot")
gdf.head(10).explore(m=m, marker_kwds={"radius": 3, "color": "red"})

In [ ]:
# merge overlapping polygons
# https://gis.stackexchange.com/questions/334459/how-to-dissolve-overlapping-polygons-using-geopandas
mergedpolys = gpd.GeoDataFrame(
    geometry=isochrones.groupby("time")["geometry"]
    .agg(lambda g: g.unary_union)
    .apply(lambda g: [g] if isinstance(g, shapely.geometry.Polygon) else g.geoms)
    .explode(),
    crs=isochrones.crs,
)

# visualize merged polygons
m = None
for i, wt in enumerate(walk_times[::-1]):
    m = mergedpolys.loc[[wt]].explore(
        m=m,
        color=colors.to_hex(cm.get_cmap("tab20b", len(walk_times))(i)),
        name=wt,
        height=300,
        width=500,
    )

m = gdf.head(10).explore(
    m=m, marker_kwds={"radius": 3, "color": "red"}, name="schools"
)
folium.LayerControl().add_to(m)

m

In [ ]:
austin = ox.geocode_to_gdf("Austin, Texas")

In [ ]:
ax = austin.plot(color='white', edgecolor='black', figsize=(16,12))
gdf.plot(ax=ax, color='red', markersize=2)